<a href="https://colab.research.google.com/github/google-ai-edge/LiteRT/blob/main/tests/flex_delegate/tflite_with_tf_ops.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2024 The AI Edge LiteRT Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Model Conversion

Model conversion with `SELECT_TF_OPS`.
This example shows a way to convert TF model with using TF ops.

In [1]:
import tensorflow as tf

@tf.function(
  input_signature=[tf.TensorSpec(shape=[1], dtype=tf.float32)])
def model(x):
  return tf.sinh(x)

print(model([1.0]))

converter = tf.lite.TFLiteConverter.from_concrete_functions([model.get_concrete_function()], model)
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS    # enable TensorFlow ops.
]
tflite_model = converter.convert()

with open('model_sinh.tflite', 'wb') as f:
  f.write(tflite_model)



tf.Tensor([1.1752012], shape=(1,), dtype=float32)


## Run the model with tf.lite

Run the converted `model_sinh.tflite` model with `tf.lite.Interpreter`

In [2]:
import tensorflow as tf

interpreter = tf.lite.Interpreter('model_sinh.tflite')
interpreter.allocate_tensors()

output_details = interpreter.get_output_details()[0]
input_details = interpreter.get_input_details()[0]
input_data = tf.constant(1.0, shape=[1])
interpreter.set_tensor(input_details['index'], input_data)
interpreter.invoke()
res = interpreter.get_tensor(output_details['index'])
print(res)

[1.1752012]


## Run the model with tflite_runtime


Run the converted `model_sinh.tflite` model with `tflite_runtime.interpreter.Interpreter`
This script doesn't use `tf` package. It uses smaller `tflite_runtime` package with `numpy` package.

In [3]:
# Install tflite_runtime PIP package
!pip install tflite_runtime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 26.2 MB/s eta 0:00:00


In [4]:
import tflite_runtime.interpreter as tflite
import numpy as np

interpreter = tflite.Interpreter('model_sinh.tflite')
interpreter.allocate_tensors()

output_details = interpreter.get_output_details()[0]
input_details = interpreter.get_input_details()[0]
input_data = np.array([1.0], dtype=np.float32)
interpreter.set_tensor(input_details['index'], input_data)
interpreter.invoke()
res = interpreter.get_tensor(output_details['index'])
print(res)

[1.1752012]


## Run the model with LiteRT


Run the converted `model_sinh.tflite` model with `ai_edge_litert.interpreter.Interpreter`.

In [5]:
# Install LiteRT package
!pip install ai-edge-litert

In [7]:
# Run inference with LiteRT
from ai_edge_litert.interpreter import Interpreter
import numpy as np

interpreter = Interpreter('model_sinh.tflite')
interpreter.allocate_tensors()

output_details = interpreter.get_output_details()[0]
input_details = interpreter.get_input_details()[0]
input_data = np.array([1.0], dtype=np.float32)
interpreter.set_tensor(input_details['index'], input_data)
interpreter.invoke()
res = interpreter.get_tensor(output_details['index'])
print(res)

[1.1752012]
